In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory


In [9]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [10]:
def create_chat_session():
    # Initialize the language model
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    
    # Initialize memory
    memory = ConversationBufferMemory(return_messages=True)
    
    # Create a prompt template with memory
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant who responds in a friendly and concise manner."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    # Create a chain with memory
    chain = (
        {"input": lambda x: x["input"], "history": lambda x: x["history"]}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return chain, memory

def main():
    print("Welcome to Simple LangChain Chat!")
    print("Type 'exit' to end the conversation.")
    print("-" * 50)
    
    # Create the chat chain and memory
    chat_chain, memory = create_chat_session()
    
    # Initialize conversation history
    conversation_history = []
    
    while True:
        # Get user input
        user_input = input("\nYou: ")
        
        # Check if user wants to exit
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("\nAssistant: Goodbye! Have a great day!")
            break
        
        try:
            # Add user message to history
            conversation_history.append(HumanMessage(content=user_input))
            
            # Process the user input through the chain
            response = chat_chain.invoke({
                "input": user_input,
                "history": conversation_history
            })
            
            # Add AI response to history
            conversation_history.append(AIMessage(content=response))
            
            # Display the response
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"\nError: {str(e)}")


In [11]:
main()

/var/folders/_y/jx3g_j6n7bl_txk7_1tfhq200000gn/T/ipykernel_56431/6343565.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


Welcome to Simple LangChain Chat!
Type 'exit' to end the conversation.
--------------------------------------------------

Assistant: The product of 3 and 4 is 12. ✨

Assistant: The product of 12 and 5 is 60. 🌟

Assistant: Goodbye! Have a great day!
